### Yangyang Dai
### PS 7

(a) Use sklearn.linear model.LogisticRegression to fit a logistic model of mpg high on features number of cylinders (cyl), engine displacement (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehi- cle year (yr), vehicle origin (orgn). 

In [23]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [24]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto = auto.dropna()

In [25]:
auto['mpg_median'] = auto['mpg'].median()
auto['mpg_high'] = auto['mpg'] > auto['mpg_median']
auto['mpg_high'] = auto['mpg_high'].apply(int)
auto['const'] = 1

In [26]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_median,mpg_high,const
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,22.75,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,22.75,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,22.75,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,22.75,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,22.75,0,1


In [27]:
X = auto[['const', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

In [28]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)
Logit_err_0 = np.zeros(k)
Logit_err_1 = np.zeros(k)
Logit_MSE_VEC = np.zeros(k)
k_ind = int(0)

In [29]:
for train_index, test_index in kf_log.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    true_vals = np.asarray(true_vals)
    true_preds = np.asarray(true_preds)
    false_vals = np.asarray(false_vals)
    false_preds = np.asarray(false_preds)
    true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = True is', true_MSE_kf[k_ind])
    print('MSE for mpg_high = False is', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('error rate for mpg_high = True', true_MSE,
      'error rate for mpg_high = False', false_MSE)

k index= 0
MSE for test set 0  is 0.0918367346939
MSE for mpg_high = True is 0.130434782609
MSE for mpg_high = False is 0.0576923076923
k index= 1
MSE for test set 1  is 0.102040816327
MSE for mpg_high = True is 0.0816326530612
MSE for mpg_high = False is 0.122448979592
k index= 2
MSE for test set 2  is 0.132653061224
MSE for mpg_high = True is 0.115384615385
MSE for mpg_high = False is 0.152173913043
k index= 3
MSE for test set 3  is 0.102040816327
MSE for mpg_high = True is 0.145454545455
MSE for mpg_high = False is 0.046511627907
test estimate MSE k-fold= 0.107142857143 test estimate MSE standard err= 0.015306122449
error rate for mpg_high = True 0.118226649127 error rate for mpg_high = False 0.0947067070586


(b) Use sklearn.ensemble.RandomForestClassifier to fit a random for- est model of mpg high on max features=2 out of the seven possible fea- tures used in part (a). Set n estimators=20, set bootstrap=True, set oob score=True, and set random state=25. Report the MSE of the ran- dom forest model as the MSE from the .oob prediction object, and re- port the error rates for each category of mpg high from the .oob prediction object.

In [30]:
auto_tree = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
auto_tree.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import classification_report

oob_prediction = auto_tree.oob_decision_function_.T[1]
MSE_RF = pd.DataFrame({'pred' : oob_prediction, 'yvals': y})
MSE_RF['pred'] = MSE_RF['pred'].apply(lambda x: 1 if x >= 0.5 else 0)
print(classification_report(MSE_RF['yvals'], MSE_RF['pred']))
error = (MSE_RF['pred'] != MSE_RF['yvals']).apply(int)
RF_err_0 = ((MSE_RF['pred'] == 0) * error).sum() / (MSE_RF['pred'] == 0).sum() 
RF_err_1 = ((MSE_RF['pred'] == 1) * error).sum() / (MSE_RF['pred'] == 1).sum() 
print('The error rate(mpg_high = True:) is', RF_err_0.mean())
print('The error rate(mpg_high = False:) is', RF_err_1.mean())
print('The MSE of the model is', error.mean())

             precision    recall  f1-score   support

          0       0.93      0.91      0.92       196
          1       0.91      0.93      0.92       196

avg / total       0.92      0.92      0.92       392

The error rate(mpg_high = True:) is 0.0725388601036
The error rate(mpg_high = False:) is 0.0854271356784
The MSE of the model is 0.07908163265306123


(c) Use sklearn.svm.SVC to fit a support vector machines model of mpg high with a Gaussian radial basis function kernel kernel=’rbf’ on the seven features used in part (a). Set the penalty parameter to C=1 and set gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds exactly as in part (a).

In [37]:
k = 4
kf_svm = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svm.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
true_MSE_kf = np.zeros(k)
false_MSE_kf = np.zeros(k)

k_ind = int(0)

In [38]:
for train_index, test_index in kf_log.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = svm.SVC(kernel='rbf', C=1, gamma=.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    
    if true_vals and true_preds:
        true_vals = np.asarray(true_vals)
        true_preds = np.asarray(true_preds)
        
        true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    else:
        true_MSE_kf[k_ind] = 0.0
    
    if false_vals and false_preds:
        false_preds = np.asarray(false_preds)
        false_vals = np.asarray(false_vals)
        
        false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
    else:
        false_MSE_kf[k_ind] = 0.0
            
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = True is', true_MSE_kf[k_ind])
    print('MSE for mpg_high = False is', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('error rate for mpg_high = True', true_MSE,
      'error rate for mpg_high = False', false_MSE)

k index= 0
MSE for test set 0  is 0.540816326531
MSE for mpg_high = True is 0.552083333333
MSE for mpg_high = False is 0.0
k index= 1
MSE for test set 1  is 0.520408163265
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.520408163265
k index= 2
MSE for test set 2  is 0.520408163265
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.53125
k index= 3
MSE for test set 3  is 0.448979591837
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.47311827957
test estimate MSE k-fold= 0.507653061224 test estimate MSE standard err= 0.0348846794163
error rate for mpg_high = True 0.138020833333 error rate for mpg_high = False 0.381194110709


(d) Which of the above three models do you think is the best predictor of mpg high? Why?

Based on the results from above, we can see that the RandomForestClassifier is the most accurate model.  It has the smallest aggregate MSE and smallest MSE for each category of mpg_high. The LogisticRegression model also has relatively small MSE values. The SVM model is the least accurate model among all the models.  